In [4]:
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

In [5]:
pretrained_model_generator, input_encoder = load_pretrained_model()

 Local model dump file /home/matt/proteinbert_models/default.pkl doesn't exist. Will download https://media.githubusercontent.com/media/nadavbra/proteinbert_data_files/master/epoch_92400_sample_23500000.pkl into /home/matt/proteinbert_models. Please approve or reject this (to exit and potentially call the function again with different parameters).


Do you approve downloading the file into the specified directory? Please specify "Yes" or "No": Yes


Downloaded file: /home/matt/proteinbert_models/epoch_92400_sample_23500000.pkl
Created: /home/matt/proteinbert_models/default.pkl


In [6]:
model = pretrained_model_generator.create_model(512) # seq_len

2024-05-06 16:12:15.749361: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-05-06 16:12:15.750373: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-05-06 16:12:15.757369: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-06 16:12:15.757564: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.725GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2024-05-06 16:12:15.757652: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such f

In [22]:
prots = {
    "test": "MLGMIRNSLFGSVETWPWQVLSTGGKEDVSYEERACEGGKFATVEVTDKPVDEALREAMPKIMKYVGGTN",
}

prot_seqs = []
for name, seq in prots.items():
    seq_tok, global_anno = input_encoder.encode_X([seq], 512)

In [23]:
seq_pred, anno_pred = model.predict([seq_tok, global_anno])

In [31]:
seq_pred_str = pred_to_str(seq_pred.argmax(axis=2))

In [32]:
def pred_to_str(pred):
    idx_to_tok = list("ACDEFGHIKLMNPQRSTUVWXY?^$_")
    ret = []
    for i in pred:
        out = "".join([idx_to_tok[j] for j in i])
        ret.append(out)

    return ret

In [33]:
seq_pred_str[0]

'^MLGMIRNSLFGSVETWPWQVLSTGGKEDVSYEERACEGGKFATVEVTDKPVDEALREAMPKIMKYVGGTN$$$$KLLLLLLLLLLLLLLL$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$LLLS'

In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input-annotations (InputLayer)  [(None, 8943)]       0                                            
__________________________________________________________________________________________________
input-seq (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
dense-global-input (Dense)      (None, 512)          4579328     input-annotations[0][0]          
__________________________________________________________________________________________________
embedding-seq-input (Embedding) (None, 512, 128)     3328        input-seq[0][0]                  
______________________________________________________________________________________________

In [47]:
def print_layer(name, model=model):
    weights = model.get_layer(name).get_weights()
    for i, w in enumerate(weights):
        print(i)
        print(w.shape)
        print(w)
        print("---")

In [48]:
print_layer("global-to-seq-dense-block1")

0
(512, 128)
[[ 0.06834244  0.01988265  0.01552765 ...  0.05288291  0.00152298
   0.1018758 ]
 [ 0.01048297 -0.05605717 -0.12701117 ... -0.06786665 -0.15304331
  -0.05094445]
 [ 0.01449823  0.03993478  0.2529434  ...  0.275232    0.19020355
   0.1426012 ]
 ...
 [ 0.05103726 -0.19401127 -0.08706205 ...  0.03590778  0.03056292
  -0.01719012]
 [-0.09253843 -0.19104876  0.08978942 ... -0.00519278 -0.07405298
  -0.1878654 ]
 [-0.01260577  0.00932141 -0.04474317 ...  0.05701987  0.0150525
   0.1956772 ]]
---
1
(128,)
[ 2.04922026e-03 -1.17570872e-03 -3.31911421e-03 -8.76402110e-03
 -1.40250858e-03 -1.70294530e-04 -1.28561968e-03 -2.20698491e-03
 -5.81347570e-03  2.20037461e-03 -1.44005299e-03 -2.72850553e-03
  1.16078136e-03  1.75407482e-03  2.24006432e-03  1.71314110e-03
 -5.13397343e-03 -1.37390476e-03 -3.93479923e-03  1.34350162e-03
  5.04005409e-04 -2.47267802e-04 -7.82131683e-04 -3.62749095e-03
  4.95223736e-04 -1.06377306e-03 -7.50516192e-04 -2.57725292e-03
  5.27181604e-04 -4.55610873